### Extract Data and Clean It

#### 1. Import libraries and set options

In [1]:
import os
import pandas as pd
from IPython.display import display
from fuzzywuzzy import process
pd.set_option('max_colwidth', 400)

/home/jonny/miniconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


#### 2. Create Dataframes and clean data
##### 2.1 Match data Dataframe
Collate csv files, convert into lists and create first dataframe containing result data.

In [2]:
project_dir = os.path.dirname(os.path.abspath(''))
data_dir = os.path.join(project_dir, 'raw_data', 'dataset_1')

field_names = []
df_list = []

for root, _, files in os.walk(data_dir):
    for filenames in files:
        file_path = os.path.join(root, filenames)
        if field_names == []:
            field_names = pd.read_csv(file_path, nrows=0).columns.tolist()
        else:
            new_field_names = pd.read_csv(file_path, nrows=0).columns.tolist()
            for index, element in enumerate(field_names):
                if element != new_field_names[index]:
                    print(f"Field names don't match in {filenames}")
                    break   
        df_list.extend(pd.read_csv(file_path).values.tolist())

results_df = pd.DataFrame(df_list, columns=field_names)
display(results_df.head())

results_df.info()

,Home_Team,Away_Team,Result,Link,Season,Round,League
0,Perugia,Bologna,2-1,https://www.besoccer.com/match/perugia/bologna/2015164038,2015,1,serie_b
1,Avellino,Pro Vercelli,1-0,https://www.besoccer.com/match/us-avellino/us-pro-vercelli-calcio/2015164030,2015,1,serie_b
2,Catania,Virtus Lanciano,3-3,https://www.besoccer.com/match/catania/ss-virtus-lanciano-1924/2015164031,2015,1,serie_b
3,Crotone,Ternana Calcio,0-2,https://www.besoccer.com/match/fc-crotone/ternana-calcio/2015164032,2015,1,serie_b
4,Virtus Entella,SSC Bari,0-2,https://www.besoccer.com/match/virtus-entella/as-bari/2015164033,2015,1,serie_b


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146498 entries, 0 to 146497
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Home_Team  146498 non-null  object
 1   Away_Team  146498 non-null  object
 2   Result     146498 non-null  object
 3   Link       146498 non-null  object
 4   Season     146498 non-null  int64 
 5   Round      146498 non-null  int64 
 6   League     146498 non-null  object
dtypes: int64(2), object(5)
memory usage: 7.8+ MB


**Findings:**
Based on the above, the results, season, round and league need to be validated and if applicable cleaned.

Team names and links will have to be assumed to be correct for now.

**Results** - Check scores validity and update where required for consistency.

In [3]:
possible_results = []
for i in range(20):
    for j in range(20):
        possible_results.append(f'{i}-{j}')
display(results_df.loc[~results_df['Result'].isin(possible_results)])

results_df.iloc[1118, 2] = '3-2'
results_df.iloc[1120, 2] = '0-1'
results_df.iloc[45638, 2] = '1-1'
results_df.iloc[91355, 2] = '0-0'
results_df.iloc[119078, 2] = '0-0'
results_df = results_df.drop([133873]) # This match was cancelled
results_df.loc[~results_df['Result'].isin(possible_results)]

,Home_Team,Away_Team,Result,Link,Season,Round,League
1118,Spezia,SSC Bari,3 (3-2) 2,https://www.besoccer.com/match/asd-spezia/as-bari/2013228635,2013,18,serie_b
1120,Cittadella,Varese,0 (0-1) 1,https://www.besoccer.com/match/as-cittadella/as-varese-1910/2013228628,2013,18,serie_b
45638,Marítimo,Sporting CP,1 (0-0) 1,https://www.besoccer.com/match/maritimo/sporting-lisbon/20139929,2013,3,primeira_liga
91355,Aston Villa,Everton,17 JAN,https://www.besoccer.com/match/aston-villa-fc/everton-fc/202172414,2021,19,premier_league
119078,PEC Zwolle,VVV Venlo,0 (0-0) 0,https://www.besoccer.com/match/fc-zwolle/vvv/20136495,2013,15,eredivisie
133873,Strasbourg,PSG,18 MAR,https://www.besoccer.com/match/strasbourg/paris-saint-germain-fc/202010330,2020,28,ligue_1


,Home_Team,Away_Team,Result,Link,Season,Round,League


**Team Names** - Confirm that there are no spurious/mis spelt team names (i.e. appearing less than 10 times).

In [4]:
display(results_df[results_df.groupby('Home_Team')['Home_Team'].transform('size') < 10])

,Home_Team,Away_Team,Result,Link,Season,Round,League


**Season, Round, League** - Confirm that the set of values is consistent and valid.

In [5]:
print(set(results_df['Season']))
print(set(results_df['League']))
print(set(results_df['Round']))

{1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021}
{'ligue_2', 'eredivisie', '2_liga', 'bundesliga', 'championship', 'primeira_liga', 'premier_league', 'serie_a', 'ligue_1', 'segunda_liga', 'serie_b', 'primera_division', 'eerste_divisie', 'segunda_division'}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46}


##### 2.2 Match Info Dataframe
Convert csv files into dataframe containing match data.

In [6]:
data_dir = os.path.join(project_dir, 'raw_data', 'dataset_2')
match_csv = os.path.join(data_dir, 'Match_Info.csv')
match_df = pd.read_csv(match_csv)
display(match_df.head())
match_df.info()

,Link,Date_New,Referee,Home_Yellow,Home_Red,Away_Yellow,Away_Red
0,/match/saarbrucken/stuttgarter-kickers/1990,"Saturday, 29 July 1989, 15:00",\r\nReferee: Hans-Jürgen Weber\r\n,0.0,0.0,3.0,0.0
1,/match/sc-freiburg/unterhaching/1990,"Saturday, 29 July 1989, 15:00",\r\nReferee: Kurt Wittke\r\n,1.0,0.0,0.0,0.0
2,/match/vfl-osnabruck/meppen/1990,"Saturday, 29 July 1989, 15:00",\r\nReferee: Werner Föckler\r\n,3.0,0.0,2.0,0.0
3,/match/rot-weiss-essen/schalke-04/1990,"Saturday, 29 July 1989, 15:00",\r\nReferee: Heinz Werner\r\n,2.0,0.0,2.0,0.0
4,/match/alemannia-aachen/msv-duisburg/1990,"Saturday, 29 July 1989, 15:00",\r\nReferee: Hans-Peter Dellwing\r\n,1.0,0.0,1.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143348 entries, 0 to 143347
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Link         143348 non-null  object 
 1   Date_New     143348 non-null  object 
 2   Referee      143348 non-null  object 
 3   Home_Yellow  122798 non-null  float64
 4   Home_Red     122798 non-null  float64
 5   Away_Yellow  122798 non-null  float64
 6   Away_Red     122798 non-null  float64
dtypes: float64(4), object(3)
memory usage: 7.7+ MB


**Findings:**
- Based on the above, the links are incomplete compared to the results df and will need manipulating so that the dfs can be joined.
- Card numbers need to be validated. There are several matches in which this dataset is incomplete. These will have to be left (approx 20k have null values)
- Referee strings need to be cleaned.
- Links need to be cleaned to match those in results_df.

**Cards** - Validate numbers of cards.

In [7]:
print(set(match_df.loc[~match_df['Home_Yellow'].isna(), 'Home_Yellow']))
print(set(match_df.loc[~match_df['Home_Red'].isna(), 'Home_Red']))
print(set(match_df.loc[~match_df['Away_Yellow'].isna(), 'Away_Yellow']))
print(set(match_df.loc[~match_df['Away_Red'].isna(), 'Away_Red']))

{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0}
{0.0, 1.0, 2.0, 3.0}
{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0}
{0.0, 1.0, 2.0, 3.0, 4.0}


**Referee** - Clean up referee strings

In [8]:
match_df['Referee'] = match_df['Referee'].replace('\r\n', '', regex=True)
display(match_df[match_df['Referee'].str.contains('\r\n')])
match_df.head()

,Link,Date_New,Referee,Home_Yellow,Home_Red,Away_Yellow,Away_Red


,Link,Date_New,Referee,Home_Yellow,Home_Red,Away_Yellow,Away_Red
0,/match/saarbrucken/stuttgarter-kickers/1990,"Saturday, 29 July 1989, 15:00",Referee: Hans-Jürgen Weber,0.0,0.0,3.0,0.0
1,/match/sc-freiburg/unterhaching/1990,"Saturday, 29 July 1989, 15:00",Referee: Kurt Wittke,1.0,0.0,0.0,0.0
2,/match/vfl-osnabruck/meppen/1990,"Saturday, 29 July 1989, 15:00",Referee: Werner Föckler,3.0,0.0,2.0,0.0
3,/match/rot-weiss-essen/schalke-04/1990,"Saturday, 29 July 1989, 15:00",Referee: Heinz Werner,2.0,0.0,2.0,0.0
4,/match/alemannia-aachen/msv-duisburg/1990,"Saturday, 29 July 1989, 15:00",Referee: Hans-Peter Dellwing,1.0,0.0,1.0,0.0


Check links in the results df are in the match_df by standardising link string.

In [9]:
results_df['Link'] = results_df['Link'].apply(lambda x: x[:(x.rfind('/') + 5)])
match_df['Link'] = 'https://www.besoccer.com' + match_df['Link']
match_df['Link'] = match_df['Link'].replace('match_\w+/', 'match/', regex=True)

##### 2.3 Team Info Dataframe
Convert csv files into dataframe containing team info data.

In [10]:
data_dir = os.path.join(project_dir, 'raw_data', 'dataset_2')
team_csv = os.path.join(data_dir, 'Team_Info.csv')
team_df = pd.read_csv(team_csv)
display(team_df.head())
print(team_df.info())

,Team,City,Country,Stadium,Capacity,Pitch
0,Wattenscheid 09,Bochum-Wattenscheid,Germany,Lohrheidestadion,16233,Natural
1,Hertha BSC,Berlín,Germany,Olympiastadion Berlin,76065,Natural
2,Unterhaching,Unterhaching,Germany,Sportpark Unterhaching,15053,Natural
3,Fortuna Köln,Cologne,Germany,Südstadion,14944,Natural
4,MSV Duisburg,Duisburgo,Germany,Schauinsland-Reisen-Arena,31514,Natural


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544 entries, 0 to 543
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Team      544 non-null    object
 1   City      544 non-null    object
 2   Country   544 non-null    object
 3   Stadium   447 non-null    object
 4   Capacity  544 non-null    object
 5   Pitch     447 non-null    object
dtypes: object(6)
memory usage: 25.6+ KB
None


**Findings:**
- Based on the above, the country and pitch need to be validated and if applicable cleaned.
- City, team names, capacity and stadium will have to be assumed to be correct for now.

**Country** - Check countries are applicable and valid.

In [11]:
print(set(team_df['Country']))

{'Netherlands', 'Portugal', 'Spain', 'France', 'Germany', 'Italy', 'England'}


**Pitch** - Standardise entries for pitch type.

In [12]:
print(set(team_df['Pitch']))
list_to_update = ['cesped real', 'Grass', 'Césped Natural', 'Cesped natural', 'NATURAL', 'Natural grass', 'Césped', 'Césped natural', 'natural', 'natural grass', 'cesped natural', 'grass']
team_df.loc[team_df['Pitch'].isin(list_to_update), 'Pitch'] = 'Natural'
team_df.loc[team_df['Pitch'] == 'Césped Artificial', 'Pitch'] = 'Artificial'
print(set(team_df['Pitch']))

{nan, 'Cesped natural', 'natural', 'Artificial', 'AirFibr ', 'Césped Natural', 'Grass', 'Césped', 'Césped natural', 'cesped natural', 'Natural grass', 'natural grass', 'Césped Artificial', 'grass', 'NATURAL', 'Natural', 'cesped real'}
{nan, 'AirFibr ', 'Artificial', 'Natural'}


#### 3 Combine Datasets
##### 3.1 Compare Datasets and Clean
Find results with teams not in team_df.
Create dictionary of team names to be replaced.

In [13]:
not_found_home = set(results_df[~results_df['Home_Team'].isin(team_df['Team'])]['Home_Team'])
not_found_away = set(results_df[~results_df['Away_Team'].isin(team_df['Team'])]['Away_Team'])
print(not_found_home == not_found_away)
print(not_found_home)

True
{'SV Wacker Burghausen', 'Olympique Marseille', 'Licata', 'B. Mönchengladbach', 'Oldham Athletic AFC', 'Kickers Offenbach FC', 'Paços de Ferreira', 'SV Wehen Burghausen', 'FC Libourne Saint Seurin', 'Barletta', 'Fortuna Düsseldorf', 'SV Eintracht Trier', 'Casertana', 'Stuttgarter Kickers', 'Real Unión de Irún', 'Gimnàstic Tarragona', 'Brighton & Hove Albion', 'Blau-Weiß 1890 Berlin', 'Queens Park Rangers', 'Peterborough United', 'Calcio Portogruaro-Summaga', 'FC Carl Zeiss Jena', 'Sporting Toulon Var', '1. FC Lokomotive Leipzig', 'Taranto', 'Sheffield Wednesday', 'Vitória Guimarães B', 'Rot-Weiß Oberhausen', 'Evian Thonon Gaillard', 'West Bromwich Albion', 'CS Louhans Cuiseaux', 'Siegen Sportfreunde ', 'Eintracht Frankfurt', 'AS Lucchese Libertas 1905', 'Vitória Guimarães', 'Würzburger Kickers'}


In [14]:
team_list = list(set(team_df['Team'].to_list()))
teams_to_change = {}

for team in not_found_home:
    teams_to_change[team] = process.extractOne(team, team_list)[0]
teams_to_change

{'SV Wacker Burghausen': 'Wacker Burghausen',
 'Olympique Marseille': 'Olympique',
 'Licata': 'Alicante',
 'B. Mönchengladbach': 'Mönchengladbach',
 'Oldham Athletic AFC': 'Oldham Athletic',
 'Kickers Offenbach FC': 'Kickers Offenbach',
 'Paços de Ferreira': 'Paços Ferreira',
 'SV Wehen Burghausen': 'Wehen Burghausen',
 'FC Libourne Saint Seurin': 'FC Cartagena',
 'Barletta': 'Arles',
 'Fortuna Düsseldorf': 'Fortuna',
 'SV Eintracht Trier': 'Eintracht Trier',
 'Casertana': 'Catania',
 'Stuttgarter Kickers': 'Stuttgarter',
 'Real Unión de Irún': 'Real Unión Irún',
 'Gimnàstic Tarragona': 'Gimnàstic',
 'Brighton & Hove Albion': 'Brighton Hove Alb.',
 'Blau-Weiß 1890 Berlin': 'Blau-Weiß 1890 B.',
 'Queens Park Rangers': 'Queens Park Range.',
 'Peterborough United': 'Peterborough',
 'Calcio Portogruaro-Summaga': 'Porto',
 'FC Carl Zeiss Jena': 'Carl Zeiss Jena',
 'Sporting Toulon Var': 'Sporting Toulon',
 '1. FC Lokomotive Leipzig': 'Lokomotive Leipzig',
 'Taranto': 'Atalanta',
 'Sheffield

Pop team names that are incorrectly matched. And then update the dictionary.

In [15]:
keys_to_drop = {
    'Licata': 'Alicante',
    'Casertana': 'Catania',
    'Barletta': 'Arles',
    'Taranto': 'Atalanta',
    'Calcio Portogruaro-Summaga': 'Calcio',
    'FC Libourne Saint Seurin': 'Paris FC'}

for k in keys_to_drop.keys():
    teams_to_change.pop(k)

In [16]:
values_to_update = {"Home_Team": teams_to_change}
results_df.replace(values_to_update, inplace=True)
values_to_update = {"Away_Team": teams_to_change}
results_df.replace(values_to_update, inplace=True)
not_found_home = set(results_df[~results_df['Home_Team'].isin(team_df['Team'])]['Home_Team'])
print(not_found_home)

{'FC Libourne Saint Seurin', 'Barletta', 'Casertana', 'Licata', 'Calcio Portogruaro-Summaga', 'Taranto'}


As there are 3503 unmatched links out 146000 data entries, these unmatched links can be dropped. Matching these would otherwise be too computationally/time expensive.

###### 3.2 Merge Datasets
Merge as follows:
- Pull in team_df into results_df
- Pull in match_df into results_df

In [17]:
team_df = team_df.rename(columns={'Team' : 'Home_Team'})

In [18]:
df = pd.merge(results_df, match_df, on='Link', how='left')
df = pd.merge(df, team_df, on='Home_Team', how='left')
display(df.head())
print(df.info())

,Home_Team,Away_Team,Result,Link,Season,Round,League,Date_New,Referee,Home_Yellow,Home_Red,Away_Yellow,Away_Red,City,Country,Stadium,Capacity,Pitch
0,Perugia,Bologna,2-1,https://www.besoccer.com/match/perugia/bologna/2015,2015,1,serie_b,"Friday, 29 August 2014, 20:30",Referee: Claudio Gavillucci,4.0,0.0,4.0,2.0,Perugia,Italy,Stadio Renato Curi,28000,Natural
1,Avellino,Pro Vercelli,1-0,https://www.besoccer.com/match/us-avellino/us-pro-vercelli-calcio/2015,2015,1,serie_b,"Saturday, 30 August 2014, 20:30",Referee: Ivano Pezzuto,2.0,0.0,4.0,1.0,Avellino,Italy,Stadio Partenio,10215,Natural
2,Catania,Virtus Lanciano,3-3,https://www.besoccer.com/match/catania/ss-virtus-lanciano-1924/2015,2015,1,serie_b,"Saturday, 30 August 2014, 20:30",Referee: Daniele Minelli,6.0,1.0,3.0,1.0,Catania,Italy,Stadio Angelo Massimino,23420,Natural
3,Crotone,Ternana Calcio,0-2,https://www.besoccer.com/match/fc-crotone/ternana-calcio/2015,2015,1,serie_b,"Saturday, 30 August 2014, 20:30",Referee: Maurizio Mariani,5.0,0.0,4.0,0.0,Crotone,Italy,Ezio Scida,16640,Natural
4,Virtus Entella,SSC Bari,0-2,https://www.besoccer.com/match/virtus-entella/as-bari/2015,2015,1,serie_b,"Saturday, 30 August 2014, 20:30",Referee: Leonardo Baracani,2.0,0.0,2.0,0.0,Chiavari,Italy,Stadio Comunale Chiavari,4154,Artificial


<class 'pandas.core.frame.DataFrame'>
Int64Index: 146497 entries, 0 to 146496
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Home_Team    146497 non-null  object 
 1   Away_Team    146497 non-null  object 
 2   Result       146497 non-null  object 
 3   Link         146497 non-null  object 
 4   Season       146497 non-null  int64  
 5   Round        146497 non-null  int64  
 6   League       146497 non-null  object 
 7   Date_New     142994 non-null  object 
 8   Referee      142994 non-null  object 
 9   Home_Yellow  123024 non-null  float64
 10  Home_Red     123024 non-null  float64
 11  Away_Yellow  123024 non-null  float64
 12  Away_Red     123024 non-null  float64
 13  City         146305 non-null  object 
 14  Country      146305 non-null  object 
 15  Stadium      137036 non-null  object 
 16  Capacity     146305 non-null  object 
 17  Pitch        137362 non-null  object 
dtypes: float64(4), int64(2),

#### 4 Export Dataset

In [19]:
# Save to json file
 
df_json = df.to_json(os.path.join(project_dir, 'clean_dataset.json'))